In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot
%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# hyperparameters

input_size = 784 #28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# MNIST Data set

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=False, 
                                           transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                          shuffle=False)

In [13]:
examples = iter(train_loader)

for ii in examples:
    print(ii[0].shape, ii[1].shape)
    break

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [18]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNN, self).__init__()
        
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        
        return out

In [23]:
model = FNN(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss() # Loss fuction will apply softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# training loop

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for ii, (images, labels) in enumerate(train_loader):
        #100, 1, 28, 28
        # 100, 784
        
        images = images.reshape(-1, 28*28).to(device) # Reshape images
        labels = labels.to(device)
        
        # forward 
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (ii+1)%100 == 0:
            print(f"Epoch {epoch+1}/ {num_epochs}, step {ii+1}/{n_total_steps}, loss = {loss.item():.5f}")

Epoch 1/ 2, step 100/600, loss = 0.67910
Epoch 1/ 2, step 200/600, loss = 0.37264
Epoch 1/ 2, step 300/600, loss = 0.36900
Epoch 1/ 2, step 400/600, loss = 0.36245
Epoch 1/ 2, step 500/600, loss = 0.22123
Epoch 1/ 2, step 600/600, loss = 0.19070
Epoch 2/ 2, step 100/600, loss = 0.19016
Epoch 2/ 2, step 200/600, loss = 0.30698
Epoch 2/ 2, step 300/600, loss = 0.17098
Epoch 2/ 2, step 400/600, loss = 0.16997
Epoch 2/ 2, step 500/600, loss = 0.19106
Epoch 2/ 2, step 600/600, loss = 0.09693


In [25]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        #value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        acc = 100.0 * n_correct / n_samples
        
print(f"Accuracy = {acc}")

Accuracy = 95.0
